In [1]:
import os
from tqdm import tqdm, tqdm_notebook
import numpy as np
import pandas as pd
from scipy.misc import imread
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
tf.python.control_flow_ops = control_flow_ops

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dropout
from keras.applications.vgg16 import VGG16

from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data_path = './driving-data/'
img_rows, img_cols = 160, 320

In [3]:
def load_data():
    columns = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

    print('Loading driving log ...')

    driving_log = pd.read_csv(data_path+'driving_log.csv', names=columns)
    num_rows = len(driving_log.index)

    train_images = np.zeros((num_rows, img_rows, img_cols, 3))
    train_steering = driving_log.as_matrix(['steering'])

    for index, row in tqdm_notebook(driving_log.iterrows(), unit=' rows', total=num_rows):
        fname = os.path.basename(row['center'])
        # Normalized YUV
        train_images[index] = imread(data_path+'IMG/'+fname, False, 'RGB').astype(np.float32)/255.

    print('Loaded', num_rows, 'rows.')
    return train_images, train_steering

In [4]:
X1_train, y1_train = load_data()
X_train, X_val, y_train, y_val = train_test_split(X1_train, y1_train, test_size=0.10, random_state=10)
del X1_train, y1_train

Loading driving log ...

Loaded 3685 rows.


In [5]:
model = Sequential()
model.add(Convolution2D(16, 5, 5,border_mode='valid',
                        input_shape=(img_rows, img_cols, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Convolution2D(64, 5, 5, subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
# model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))

In [ ]:
# opt = Adam(lr=1e-3)
model.compile(optimizer='adam', loss='mse')

In [ ]:
h = model.fit(X_train, y_train, batch_size=16, nb_epoch=10,
    validation_data=(X_val, y_val), verbose=1, shuffle=True)

Train on 3316 samples, validate on 369 samples
Epoch 1/10
3316/3316 [==============================] - 15s - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0025 - val_mean_squared_error: 0.0025
Epoch 2/10
 656/3316 [====>.........................] - ETA: 11s - loss: 0.0021 - mean_squared_error: 0.0021

In [ ]:
y_pred = model.predict(X_val, batch_size=16)
np.hstack((y_pred, y_val))

In [8]:
model_json = model.to_json()
with open("my_model_003.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('my_model_003.h5')